## Layer Control

This notebook demonstrates the use of the lonbord map's `layer_control`, to control layer visibility and layer properties.

In [ ]:
from pathlib import Path

import geopandas as gpd
from palettable.colorbrewer.sequential import Blues_8

from lonboard import Map, PathLayer, PolygonLayer
from lonboard.colormap import apply_continuous_cmap

### Get data

Download data from the web and save as geoparquet so we can show some data on our Lonboard map and create a layer control.

In [ ]:
file_urls = [
    (
        "ne_10m_roads_north_america.parquet",
        "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_roads_north_america.zip",
    ),
    (
        "geoBoundariesCGAZ_ADM1.parquet",
        "https://github.com/wmgeolab/geoBoundaries/raw/main/releaseData/CGAZ/geoBoundariesCGAZ_ADM1.geojson",
    ),
    (
        "rivers_asia_37331.parquet",
        "https://storage.googleapis.com/fao-maps-catalog-data/geonetwork/aquamaps/rivers_asia_37331.zip",
    ),
]
for filename, url in file_urls:
    if Path(filename).exists() is False:
        print(f"Reading {filename} from web and saving as geoparquet.")
        gdf = gpd.read_file(url, engine="pyogrio")
        gdf.to_parquet(filename)
        del gdf
    else:
        print(f"{filename} already downloaded.")

### Read geoparquet files into geopandas dataframes

In [ ]:
boundary_df = gpd.read_parquet("geoBoundariesCGAZ_ADM1.parquet")
boundary_df = boundary_df.cx[-160:-45, 20:90] # parse data down to just some of the north western hemisphere

road_df = gpd.read_parquet("ne_10m_roads_north_america.parquet")
road_df = road_df.loc[road_df["class"]=="Interstate"] # parse data down to just interstates

river_df = gpd.read_parquet("rivers_asia_37331.parquet")
river_df = river_df.loc[river_df["MAJ_NAME"] == "Amur"] # parse data down to just Amur

### Create layers

* Create a `PolygonLayer` from the boundary dataframe that is brown with a darker brown outline, that's 1 pixel wide.

* Create a `PathLayer` from the road dataframe with a title and minimum width.

* Create a `PathLayer` from the river dataframe that uses the 'Strahler' column for setting the color and width of the lines, as well as some defaults on the width to make the more prominent rivers darker and wider on the map.

In [ ]:
boundary_layer = PolygonLayer.from_geopandas(
    boundary_df,
    title="Boundaries",
    get_fill_color=[137, 81, 41],
    get_line_color=[102, 60, 31],
    get_line_width=1,
    line_width_units="pixels",
    stroked=True,
)

road_layer = PathLayer.from_geopandas(road_df, width_min_pixels=0.8)

river_layer = PathLayer.from_geopandas(
    river_df,
    title="Rivers",
    get_color=apply_continuous_cmap(river_df["Strahler"] / 7, Blues_8),
    get_width=river_df["Strahler"],
    width_scale=3000,
    width_min_pixels=0.5,
)

### Create the Lonboard `Map` and `layer_control`

Create a lonboard map, and then create a `layer_control` with the `include_settings` parameter to True then display them both.

With `include_settings=True` we will get a layer control that includes the setttings cog, which when expanded will allow us to change some of the layer properties. Note that we did not give this layer a title, so when we make the layer control, the default title will show in the layer control.

If the user unchecks the checkbox next to the layer's name the layer's visibility will be set to False.

!!! note

    We're only adding the boundary and road layer at this point, not the river layer.  We'll add that later, and when we do we can see the layer control automatically react to the new layer being added to the map, and it will show up in our layer control.

In [ ]:
lonboard_map = Map([boundary_layer, road_layer])
lc = lonboard_map.layer_control(include_settings=True)

display(lonboard_map)
display(lc)

### Change the title of the road layer

By default the title of the layer is the layer's type.  When we change the title of the layer, it will automatically be changed in the layer control.

In [ ]:
road_layer.title = "Roads"

### Add the River layer

When we add the river layer to the map, the layer control will automatically detect the new layer, and also add it to the layer control.

When we expand the cog for the river layer we will see that the `Color` and the `Width` properties of the layer display `Custom` instead of a color picker/float widget.  
"Custom" is displayed in the layer control currently because the layer uses the values from the rows of data to render the lines.  This may change in future releases of Lonboard.

In [ ]:
lonboard_map.add_layer(river_layer, reset_zoom=True)